[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/GeoAI-Tutorials/blob/main/docs/PDFM/map_pdfm_features.ipynb)

**Mapping PDFM Features and Predicted Housing Prices**

## Useful Resources

- [Google's Population Dynamics Foundation Model (PDFM)](https://github.com/google-research/population-dynamics)
- Request access to PDFM embeddings [here](https://github.com/google-research/population-dynamics?tab=readme-ov-file#getting-access-to-the-embeddings)
- Zillow data can be accessed [here](https://www.zillow.com/research/data/)

## Installation

Uncomment and run the following cell to install the required libraries.

In [1]:
%pip install "leafmap[maplibre]" scikit-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.5/677.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━

## Import Libraries

In [2]:
import os
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from leafmap.common import evaluate_model, plot_actual_vs_predicted, download_file
import leafmap.maplibregl as leafmap

## Download Zillow Data

Download the Zillow home value data at the county level.

In [3]:
zhvi_url = "https://github.com/opengeos/datasets/releases/download/us/zillow_home_value_index_by_county.csv"
zhvi_file = "data/zillow_home_value_index_by_county.csv"

In [4]:
if not os.path.exists(zhvi_file):
    download_file(zhvi_url, zhvi_file)

Downloading...
From: https://github.com/opengeos/datasets/releases/download/us/zillow_home_value_index_by_county.csv
To: /content/data/zillow_home_value_index_by_county.csv
100%|██████████| 12.3M/12.3M [00:02<00:00, 5.01MB/s]


## Process Zillow Data

In [5]:
zhvi_df = pd.read_csv(
    zhvi_file, dtype={"StateCodeFIPS": "string", "MunicipalCodeFIPS": "string"}
)
zhvi_df.index = "geoId/" + zhvi_df["StateCodeFIPS"] + zhvi_df["MunicipalCodeFIPS"]
zhvi_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31
geoId/06037,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",06,037,206679.102841,...,855978.867131,851218.324044,847438.710237,848072.053695,851221.654671,853300.031599,856929.899269,862101.367502,868683.388483,873790.400103
geoId/17031,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,031,147770.953138,...,296336.449935,297281.192639,299300.336191,301953.165715,303791.255067,304848.668699,305508.214099,306334.939254,307007.292144,307364.376614
geoId/48201,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,108332.694683,...,277636.906153,278094.301024,279085.366023,280116.831608,280701.913405,280585.041362,280219.238295,279883.927250,279671.324691,279308.720918
geoId/04013,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",04,013,143192.734000,...,465029.414800,465567.119242,467017.617878,468789.154601,470188.977576,470357.292903,469753.783114,468639.799637,467695.338296,466966.903423
geoId/06073,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",06,073,213155.078588,...,899172.567276,903677.977515,911839.212243,922026.061494,930747.253219,935215.065410,936652.178767,936216.423469,935770.997631,935062.165130


## Request access to PDFM Embeddings

To request access to PDFM embeddings, please follow the instructions [here](https://github.com/google-research/population-dynamics?tab=readme-ov-file#getting-access-to-the-embeddings).

In [10]:
county_geojson = "data/county.geojson"
if not os.path.exists(county_geojson):
    raise FileNotFoundError("Please request the embeddings from Google")

FileNotFoundError: Please request the embeddings from Google

## Load county boundaries

In [11]:
county_gdf = gpd.read_file(county_geojson)
county_gdf.set_index("place", inplace=True)
county_gdf.head()

DataSourceError: data/county.geojson: No such file or directory

## Join home value data and county boundaries

In [ ]:
df = zhvi_df.join(county_gdf)
zhvi_gdf = gpd.GeoDataFrame(df, geometry="geometry")
zhvi_gdf.head()

In [ ]:
column = "2024-10-31"
gdf = zhvi_gdf[["RegionName", "State", column, "geometry"]]
gdf.head()

## Visualize home values in 2D

In [ ]:
m = leafmap.Map(style="liberty")
first_symbol_id = m.find_first_symbol_layer()["id"]
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title="Median Home Value",
    name="Median Home Value",
    before_id=first_symbol_id,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/df616803-87e9-4326-be8c-9de5a72a2d95)

## Visualize home values in 3D

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title="Median Home Value",
    extrude=True,
    scale_factor=3,
    before_id=first_symbol_id,
    name="Median Home Value",
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/65f085bb-b781-41b8-8408-440ed202b766)

## Load PDFM county embeddings

In [ ]:
embeddings_file_path = "data/county_embeddings.csv"

In [ ]:
embeddings_df = pd.read_csv(embeddings_file_path).set_index("place")
embeddings_df.head()

In [ ]:
df = embeddings_df.join(county_gdf)
embeddings_gdf = gpd.GeoDataFrame(df, geometry="geometry")
embeddings_gdf.head()

## Visualize PDFM features

Select any of the 329 PDFM features to visualize.

In [ ]:
column = "feature329"  # Change this to the feature you want to use
gdf = embeddings_gdf[[column, "state", "county", "geometry"]]
gdf.head()

In [ ]:
m = leafmap.Map(style="liberty")
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title=column,
    before_id=first_symbol_id,
    name=column,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/efa4983d-89d2-4dcc-9cd6-fefeee069bf7)

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    gdf,
    cmap="Blues",
    column=column,
    legend_title=column,
    before_id=first_symbol_id,
    name=column,
    extrude=True,
    scale_factor=0.00005,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/fd1b233b-2be0-47c3-b2d0-f53601020604)

## Join Zillow and PDFM Data

In [ ]:
data = zhvi_df.join(embeddings_df, how="inner")
data.head()

In [ ]:
embedding_features = [f"feature{x}" for x in range(330)]
label = "2024-10-31"  # Change this to the date you want to predict

In [ ]:
data = data.dropna(subset=[label])

## Split Train and Test Data

In [ ]:
data = data[embedding_features + [label]]
X = data[embedding_features]
y = data[label]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Fit Linear Regression Model

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Evaluate Linear Regression Model

In [ ]:
evaluation_df = pd.DataFrame({"y": y_test, "y_pred": y_pred})
metrics = evaluate_model(evaluation_df)
print(metrics)

In [ ]:
xy_lim = (0, 1_000_000)
plot_actual_vs_predicted(
    evaluation_df,
    xlim=xy_lim,
    ylim=xy_lim,
    title="Actual vs Predicted Home Values",
    x_label="Actual Home Value",
    y_label="Predicted Home Value",
)

![image](https://github.com/user-attachments/assets/a16a15c6-508b-40c9-aa57-8e98b8b8e216)

## Join predicted values with county boundaries

In [ ]:
df = evaluation_df.join(gdf)
df["difference"] = df["y_pred"] - df["y"]
evaluation_gdf = gpd.GeoDataFrame(df, geometry="geometry")
evaluation_gdf.drop(columns=["category", "color", column], inplace=True)
evaluation_gdf.head()

## Visualize actual home values

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    evaluation_gdf,
    cmap="Blues",
    column="y",
    legend_title="Actual Home Value",
    before_id=first_symbol_id,
    name="Actual Home Value",
    extrude=True,
    scale_factor=3,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/a86401d5-defd-4277-877b-c8f8c5e07651)

## Visualize predicted home values

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    evaluation_gdf,
    cmap="Blues",
    column="y_pred",
    legend_title="Predicted Home Value",
    before_id=first_symbol_id,
    name="Predicted Home Value",
    extrude=True,
    scale_factor=3,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/6bfe1c89-968e-4eef-84cc-98bcf07c8acb)

## Visualize difference between predicted and actual home values

In [ ]:
m = leafmap.Map(style="liberty", pitch=60)
m.add_data(
    evaluation_gdf,
    cmap="coolwarm",
    column="difference",
    legend_title="y_pred-y",
    before_id=first_symbol_id,
    name="Difference",
    extrude=True,
    scale_factor=3,
)
m.add_layer_control()
m

![image](https://github.com/user-attachments/assets/4ae58eb1-ab28-49e5-b352-d769a32d3e5c)